In [1]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformer_mpc import save_model_and_config, load_model_and_config, train_transformer, validate_transformer, perform_inference
from testnormalise import transform_predicted_to_binary_matrix, Transformer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB


MPS available, current device MPS


In [2]:
## Load and Generate Data 
from transformer_mpc import load_data_npz, normalize_batch
#file_path = "/Users/bobvanderwoude/Documents/Systems And Control/Afstuderen/Thesis/Transformer Model/Data_Microgrid/Data_Microgrid/Microgrid_aggregated_data_LARGE_20240307.npz"
## Generate Data
file_path = "path_to_microgrid_data"

cbuy_tensor, csell_tensor, cprod_tensor, power_res_tensor, power_load_tensor, x0_tensor, delta_transformed_tensor = load_data_npz(file_path)
net_power_load = power_load_tensor - power_res_tensor
src_data = torch.cat([
    cbuy_tensor.unsqueeze(-1),  # Adding an extra dimension for feature alignment
    csell_tensor.unsqueeze(-1),
    cprod_tensor.unsqueeze(-1),
    net_power_load.unsqueeze(-1),
    x0_tensor.unsqueeze(1).repeat(1, cbuy_tensor.size(1), 1)  # Repeating x0 across the sequence length
], dim=-1)  # Concatenate along the last dimension to combine features

#Create target data 
tgt_data = delta_transformed_tensor.long()

print("src_data shape" , src_data.shape)
print("tgt_data shape" , tgt_data.shape)



src_data shape torch.Size([16000, 48, 5])
tgt_data shape torch.Size([16000, 48])


In [3]:
## Create Training and Validation Dataset and Mini Batches
mini_batch_size = 32 
dataset_size = src_data.size(0)
val_size = int(dataset_size * 0.2) 
train_size = dataset_size - val_size

full_dataset = TensorDataset(src_data, tgt_data)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=mini_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=mini_batch_size, shuffle=False)

In [ ]:
## Create transformer configuration

config =  {'src_vocab_size': None, 
           'tgt_vocab_size': 32, 
           'd_model': 128,
           'num_heads': 8,
           'num_layers': 4,
           'd_ff': 2048, 
           'feature_dim': 5, 
           'max_seq_length': 48, # src_data.size(1),
           'dropout': 0.1, 
           'nr_epochs': 20, 
           'lr': 0.0001}
transformer = Transformer(
        src_vocab_size=config['src_vocab_size'], 
        tgt_vocab_size=config['tgt_vocab_size'], 
        d_model=config['d_model'], 
        num_heads=config['num_heads'], 
        num_layers=config['num_layers'], 
        d_ff=config['d_ff'], 
        feature_dim=config['feature_dim'], 
        max_seq_length=config['max_seq_length'], 
        dropout=config['dropout'])


In [ ]:
## Train a Transformer 
train_transformer(transformer, train_dataloader, val_dataloader, config)

In [ ]:
## Load Trained Transformers and Configurations 

transformer, config = load_model_and_config('name_of_trained_transformer.pth')





print("config = ", config)





In [ ]:
## Cascaded_transformer
from transformer_mpc import cascaded_transformer
models = []  #transformer_models_for_cascaded_transformer 
num_instances = 1000


cascaded_transformer(models4, val_dataset, num_instances)



